In [1]:
import d2l
from mxnet import autograd, gluon, init, nd
from mxnet.gluon import data as gdata, loss as gloss, nn, utils
import numpy as np
import pandas as pd
import gluonnlp as nlp
from utils.preprocessing import one_hot_encoder
from utils.preprocessing import missing_values_table


/Users/josephzeng/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
spec_feature = pd.read_csv('/Applications/files/classes_homework/Berkeley_ieor/data-x/project/feature_matrix_spec.csv')

In [3]:
missing_table = missing_values_table(spec_feature)

dump_feats = missing_table[missing_table['% of Total Values'] > 30].index.tolist()

spec_feature = spec_feature.drop(dump_feats, axis = 1)

spec_feature, _ = one_hot_encoder(spec_feature)

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Imputer

Target = spec_feature['TARGET']
ID = spec_feature['SK_ID_CURR']

dataset_temp = spec_feature.drop(['TARGET','SK_ID_CURR'], axis = 1)

imputer = Imputer(strategy = 'median')
scaler = MinMaxScaler(feature_range = (0,1))

dataset_preprocessed = imputer.fit_transform(dataset_temp)
dataset_preprocessed = scaler.fit_transform(dataset_preprocessed)
dataset_preprocessed = pd.DataFrame(dataset_preprocessed, columns = dataset_temp.columns)

dataset_preprocessed['TARGET'] = Target
dataset_preprocessed['SK_ID_CURR'] = ID

There are total 885 columns.
823 of them have missing values.


In [5]:
dataset_preprocessed = dataset_preprocessed[dataset_preprocessed.TARGET != -999]

In [8]:
X_ = dataset_preprocessed.drop(['TARGET','SK_ID_CURR'], axis=1)
y_ = dataset_preprocessed['TARGET']
#print(X_.shape, y_.shape)


In [9]:
from sklearn import model_selection
train_features, test_features, train_labels, test_labels = model_selection.train_test_split(X_, y_, test_size=0.3, random_state=42)

In [ ]:
print(train_features.shape, train_labels.shape)

print(test_features.shape, test_labels.shape)

In [10]:
train_features = nd.array(train_features)
train_labels = nd.array(train_labels)
test_features = nd.array(test_features)
test_labels = nd.array(test_labels)

In [13]:
train_features.shape

(215257, 789)

### 搭建神经网络（简单多层感知机）

tensorflow和pytorch我都不太熟，就先用mxnet搭了个baseline

In [39]:
def get_net(drop_prob1,drop_prob2):
    net = nn.Sequential()
    net.add(nn.Dense(789, activation="relu"),
            # Add a dropout layer after the first fully connected layer
            nn.Dropout(drop_prob1),
            nn.Dense(64, activation="relu"),
            # Add a dropout layer after the second fully connected layer
            nn.Dropout(drop_prob2),
            nn.Dense(2)
           )
    net.initialize(init.Normal(sigma=0.01))
    return net

    

In [40]:
net = get_net(0.2,0.2)

### 训练
 这里以交叉熵作为metrics

In [41]:

batch_size = 256
loss = gloss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.03, 'wd':0.1})
num_epochs = 10


train_iter = gdata.DataLoader(gdata.ArrayDataset(
        train_features, train_labels), batch_size, shuffle=True)
test_iter = gdata.DataLoader(gdata.ArrayDataset(
        test_features, test_labels), batch_size, shuffle=True)

d2l.train_ch3(net, train_iter, test_iter, loss, num_epochs, batch_size, None,
              None, trainer)

epoch 1, loss 0.3065, train acc 0.918, test acc 0.920
epoch 2, loss 0.2844, train acc 0.919, test acc 0.920
epoch 3, loss 0.2840, train acc 0.919, test acc 0.920
epoch 4, loss 0.2838, train acc 0.919, test acc 0.920
epoch 5, loss 0.2837, train acc 0.919, test acc 0.920
epoch 6, loss 0.2835, train acc 0.919, test acc 0.920
epoch 7, loss 0.2833, train acc 0.919, test acc 0.920
epoch 8, loss 0.2833, train acc 0.919, test acc 0.920
epoch 9, loss 0.2832, train acc 0.919, test acc 0.920
epoch 10, loss 0.2832, train acc 0.919, test acc 0.920
